In [ ]:
import numpy
import scipy.special
import matplotlib.pyplot

In [ ]:
class neuralNetwork:
    # инициализация
    def __init__(self, inputnodes,  hiddennodes, outputnodes, learningrate):
        # задать количество узлов во входном, скрытом и выходном слое
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # коэффициент обучения
        self.lr = learningrate

        # зададим веса связей
        # матрица весов между входным и скрытм слоем
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))

        # матрица весов между скрытм и выходным слоем
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        # использование сигомиды в качестве функции активации
        self.activation_function = lambda x:scipy.special.expit(x)

        pass

    # тренировка
    def train(self, inputs_list, targets_list):
        # преобразовать сисок входных значений в двумерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T

        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)

        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)

        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)

        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)

        # ошибка  = целевое значение - фактическое значение
        output_errors = targets - final_outputs

        # ошибки скрытого слоя - это ошибки output_errors
        # распределение пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = numpy.dot(self.who.T, output_errors)

        # обновить весовые коэффициенты связей между скрытым и выходным слоями
        self.who +=self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))

        # обновить весовые коэффициенты связей между скрытым и выходным слоями
        self.wih +=self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))

        pass

    # опрос
    def query(self, input_list):
        # преобразовать список входных значений
        # в двумерный массив
        inputs = numpy.array(input_list, ndmin=2).T

        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)

        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)

        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)

        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

    

In [ ]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1

# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [ ]:
# загрузить в список тестовый набор данных CSV-файла набора 
training_data_file = open("makeyourownneuralnetwork\mnist_dataset\mnist_train.csv", 'r') 
training_data_list = training_data_file.readlines()
training_data_file.close()


In [29]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # перебрать все записи в тренировочном наборе данных
    for record in training_data_list:
        # получить список значений, использу я символы запятой 
        all_values = record.split(',')
        # масштабировать и сместить входные значения
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0.01 за исключением
        # желаемого маркерного значения, равного 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] целевое маркерное значение для данной записи
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [ ]:
# загрузить в список тестовый набор данных CSV-файла набора 
test_data_file = open("makeyourownneuralnetwork\mnist_dataset\mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# тестирование нейронной сети

# журнал оценок работы сети, первоанчально он пустой
scorecard = []

# перебрать все записи в тестовом наборе даных
for record in test_data_list:
    # получить список значений из записи, используя символы
    # запятой в качестве разделителей
    all_values = record.split(',')
    # правильный ответ - первое значение
    correct_label = int(all_values[0])
   # print(correct_label, "истинный маркер")
    # масштабировать и сместить входные значений
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(inputs)
    # индекс наибольшего значения является маркерным значением
    label = numpy.argmax(outputs)
   # print(label, "ответ сети")
   # print("-----------------------------")
    # присоединить оценку ответа сети к концу списка
    if (label == correct_label):
        # в случае правильного ответа сети присоединить
        # к списку значени 
        scorecard.append(1)
    else:
        # в случае неправильного ответа сети присоединить
        # списку значение 0
        scorecard.append(0)
        pass
    
    pass

In [ ]:
print(scorecard)
#calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("эффективность = ", scorecard_array.sum() / scorecard_array.size)